In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [11]:
# Define a custom dataset
class NumberSumDataset(Dataset):
    def __init__(self, data_range=(1, 10)):
        """
        Initialize the dataset with a range of numbers.
        
        Args:
        data_range (tuple): A tuple specifying the range of numbers to use. 
                            Defaults to (1, 10), which generates numbers from 1 to 9.
        """
        self.numbers = list(range(data_range[0], data_range[1]))  # Create a list of numbers in the specified range.

    def __getitem__(self, index):
        """
        Retrieve a specific data sample by index.
        
        Args:
        index (int): The index of the sample to retrieve.
        
        Returns:
        tuple: A tuple containing:
               - A tensor with two numbers (number1, number2).
               - A tensor with their sum as the target.
        """
        # Calculate number1 by dividing the index by the length of the numbers list (floor division).
        number1 = float(self.numbers[index // len(self.numbers)])
        
        # Calculate number2 by taking the remainder when the index is divided by the length of the numbers list.
        number2 = float(self.numbers[index % len(self.numbers)])
        
        # Return a tuple:
        # - A tensor containing the pair of numbers [number1, number2].
        # - A tensor containing their sum [number1 + number2].
        return torch.tensor([number1, number2]), torch.tensor([number1 + number2])

    def __len__(self):
        """
        Return the total number of samples in the dataset.
        
        The total number of samples is the square of the number of elements in the range,
        since all combinations of pairs (number1, number2) are considered.
        
        Returns:
        int: Total number of samples in the dataset.
        """
        return len(self.numbers) ** 2  # Total combinations of number1 and number2.

# Example usage
dataset = NumberSumDataset(data_range=(1, 5))  # Dataset for numbers 1 through 4.
print(f"Dataset size: {len(dataset)}")  # Should print 16 (4x4).
print("First sample:", dataset[0])  # Access the first sample.

Dataset size: 16
First sample: (tensor([1., 1.]), tensor([2.]))


In [12]:
dataset = NumberSumDataset(data_range=(1, 100))

for i in range(5):
    print(dataset[i])

(tensor([1., 1.]), tensor([2.]))
(tensor([1., 2.]), tensor([3.]))
(tensor([1., 3.]), tensor([4.]))
(tensor([1., 4.]), tensor([5.]))
(tensor([1., 5.]), tensor([6.]))


### Define a simple model 

In [13]:
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.hidden_layer = nn.Linear(input_size, 128)
        self.output_layer = nn.Linear(128, 1)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.activation(self.hidden_layer(x))
        return self.output_layer(x)

### Instantitate Componets needed for Training

In [14]:
dataset = NumberSumDataset(data_range=(0, 100))
dataloader = DataLoader(dataset, batch_size=100, shuffle=True)
model = MLP(input_size=2)
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### Create a Training Loop

In [15]:
for epoch in range(10):
    total_loss = 0.0
    for number_pairs, sums in dataloader:  # Iterate over the batches
        predictions = model(number_pairs)  # Compute the model output
        loss = loss_function(predictions, sums)  # Compute the loss
        loss.backward()  # Perform backpropagation
        optimizer.step()  # Update the parameters
        optimizer.zero_grad()  # Zero the gradients

        total_loss += loss.item()  # Add the loss for all batches

    # Print the loss for this epoch
    print("Epoch {}: Sum of Batch Losses = {:.5f}".format(epoch, total_loss))

Epoch 0: Sum of Batch Losses = 400319.61770
Epoch 1: Sum of Batch Losses = 7226.86135
Epoch 2: Sum of Batch Losses = 2184.78317
Epoch 3: Sum of Batch Losses = 177.32990
Epoch 4: Sum of Batch Losses = 11.13834
Epoch 5: Sum of Batch Losses = 4.59844
Epoch 6: Sum of Batch Losses = 1.34245
Epoch 7: Sum of Batch Losses = 1.02086
Epoch 8: Sum of Batch Losses = 0.91045
Epoch 9: Sum of Batch Losses = 0.83761


In [16]:
# Test the model on 3 + 7
model(torch.tensor([3.0, 7.0]))
# tensor([10.1067], grad_fn=<AddBackward0>)

tensor([10.0723], grad_fn=<AddBackward0>)